In [ ]:
import os
from datetime import datetime
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import  BeautifulSoup
import re
from collections import OrderedDict
import pandas as pd

In [ ]:
#########################################################################
# busca todos os arquivos período explícito sem especificar a data
#########################################################################


def build_path(subfolder):
    current_folder = os.getcwd()
    folderpath = os.path.join(current_folder, subfolder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath): os.makedirs(folderpath)
    return folderpath

build_path('nomes_arquivos')
build_path('arquivos')

driver = webdriver.Chrome()
driver.implicitly_wait(0.5)
url = 'https://sip.sgb.gov.br/sip/login.php?sigla_orgao_sistema=CPRM&sigla_sistema=SEI&infra_url=L3NlaS8='
driver.get(url)

# login page

user = driver.find_element("xpath", '//*[@id="txtUsuario"]')
password = driver.find_element("xpath", '//*[@id="pwdSenha"]')
submit_button = driver.find_element("xpath", '//*[@id="sbmLogin"]')

user.send_keys('DIGITE AQUI O LOGIN')
password.send_keys('DIGITE AQUI A SENHA')
submit_button.click()

# home page

searching = driver.find_element("xpath", '//*[@id="main-menu"]/li[5]/a')
searching.click()

# search page

doc_type = driver.find_element("xpath", '//*[@id="selSeriePesquisa"]')
doc_date_explicit = driver.find_element("xpath", '//*[@id="optPeriodoExplicito"]')
sub_button = driver.find_element("xpath", '//*[@id="sbmPesquisar"]')

doc_type.send_keys('REMA - Empréstimo de Materiais ou Ex. Geológicos')
doc_date_explicit.click()
sub_button.click()

#################################################
# getting files

def get_files():

    original_window = driver.current_window_handle 
    page_docs_search = driver.find_element("xpath", '//*[@id="conteudo"]')
    page_docs = page_docs_search.text.split('Atividade Fim: ')[1:]

    file_name = []
    for i in range(len(page_docs)):
        a = page_docs[i].split('(REMA - Empréstimo de Materiais ou Ex. Geológicos) ')[1].split('\n1')[0]
        file_name.append(a)

    links = []
    for i in range(1, len(page_docs)+1):
        a = driver.find_element("xpath", '//*[@id="conteudo"]/table[%s]/tbody/tr[1]/td[1]/a[2]' %i)
        link = a.get_attribute('href')
        links.append(link)
    
    index = -1
    for i in links:
        index +=1
        name = file_name[index]
        driver.switch_to.new_window('tab')
        driver.get(i)
        with open('./arquivos/'f'processo{name}.html', 'w', encoding='iso-8859-1') as file:
                file.write(driver.page_source)        
        driver.close()
        driver.switch_to.window(original_window)
  

#################################################

# pagination 

pages = driver.find_element(by=By.CLASS_NAME, value="paginas")
list_pages = pages.text.split(' ')[:-1]

get_files()

for i in range(1, len(list_pages)):
    
    try:
        next_page = driver.find_element("xpath", '//*[@id="conteudo"]/div[2]/a[%s]' %i)
        next_page.click()
        time.sleep(6)
        get_files()
    except (NameError, TypeError):
        pass
    time.sleep(6)  
    

driver.close()
driver.quit()